In [0]:
from __future__ import print_function

import os
import pandas as pd
import numpy as np

from tensorflow.keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense
from tensorflow.keras.models import Model, load_model

In [0]:
from __future__ import print_function

import sys
import os
import pandas as pd
import numpy as np

from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense
from keras.models import Model, load_model

# # To import 'katakana' from relative path
# sys.path.append(os.path.abspath(os.path.join('..')))
# from katakana import encoding

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

In [0]:
data = pd.read_csv('/content/joined_titles.csv', header=None)
data = data.sample(frac=1, random_state=0)

print(len(data))
print(data[0:5])

53535
                      0              1
44078  Ferguson Jenkins  ファーガソン・ジェンキンス
6504      Jimmi Simpson       ジミ・シンプソン
42607       Discography       ディスコグラフィ
46551           Findlay         フィンドレー
12674         Joe Ruddy       ジョゼフ・ラディ


In [0]:
data_input = [s.lower() for s in data[0]]
data_output = [s.lower() for s in data[1]]

print(data_input[0:3])
print(data_output[0:3])

['ferguson jenkins', 'jimmi simpson', 'discography']
['ファーガソン・ジェンキンス', 'ジミ・シンプソン', 'ディスコグラフィ']


In [0]:
data_size = len(data)

training_split_index = int(data_size*60/100)
validation_split_index = int(data_size*70/100)

# We will use the first 0-60th %-tile (60%) of data for the training
training_input  = data_input[:training_split_index]
training_output = data_output[:training_split_index]

# We will use the first 60-70th %-tile (10%) of data for the validation
validation_input = data_input[training_split_index:validation_split_index]
validation_output = data_output[training_split_index:validation_split_index]

print(len(training_input))
print(len(validation_input))

32121
5353


In [0]:
START_CHAR_CODE = 1

def encode_characters(titles):
    count = 2
    encoding = {}
    decoding = {1: 'START'}
    for c in set([c for title in titles for c in title]):
        encoding[c] = count
        decoding[count] = c
        count += 1
    return encoding, decoding, count


english_encoding, english_decoding, english_dict_size = encode_characters(data_input)
japanese_encoding, japanese_decoding, japanese_dict_size = encode_characters(data_output)


print('English character dict size:', english_dict_size)
print('Katakana character dict size:', japanese_dict_size)

print(list(english_encoding.items())[0:5])
print(list(japanese_encoding.items())[0:5])

English character dict size: 52
Katakana character dict size: 88
[('ļ', 2), ('f', 3), ('1', 4), ('7', 5), ('x', 6)]
[('キ', 2), ('ケ', 3), ('コ', 4), ('リ', 5), ('ヰ', 6)]


In [0]:
training_input[0]

'ferguson jenkins'

In [0]:
training_input[0][3]

'g'

In [0]:
english_encoding[training_input[0][0]]

3

In [0]:
def transform(encoding, data, vector_size):
    transformed_data = np.zeros(shape=(len(data), vector_size), dtype='int')
    for i in range(len(data)):
        for j in range(min(len(data[i]), vector_size)):
            transformed_data[i][j] = encoding[data[i][j]]
    return transformed_data

INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoded_training_input = transform(english_encoding, training_input, vector_size=INPUT_LENGTH)
encoded_training_output = transform(japanese_encoding, training_output, vector_size=OUTPUT_LENGTH)
encoded_validation_input = transform(english_encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(japanese_encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('input', encoded_training_input)
print('output', encoded_training_output)

input [[ 3 20 19 ...  0  0  0]
 [17 24 15 ...  0  0  0]
 [26 24 36 ...  0  0  0]
 ...
 [11 13 24 ...  0  0  0]
 [26 20 34 ...  0  0  0]
 [20 22 10 ...  0  0  0]]
output [[36 52 20 ...  0  0  0]
 [80 31 48 ...  0  0  0]
 [39 59 82 ...  0  0  0]
 ...
 [36 59  5 ...  0  0  0]
 [39 59 20 ...  0  0  0]
 [23 20 34 ...  0  0  0]]


In [0]:
encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [0]:
encoder = Embedding(english_dict_size, 64, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(64,return_sequences=True, unroll=True)(encoder)

print(encoder.get_shape())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(?, 20, 64)


In [15]:
encoder.shape

TensorShape([Dimension(None), Dimension(20), Dimension(64)])

In [1]:
decoder = Embedding(japanese_dict_size, 64, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)

decoder = LSTM(64, return_sequences=True)(decoder, initial_state=[encoder, encoder])
decoder = TimeDistributed(Dense(japanese_dict_size, activation="softmax"))(decoder)

print(decoder.get_shape())

NameError: ignored

In [0]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[decoder])
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 20, 64)       3456        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 20, 64)       5696        input_2[0][0]                    
____________________________________________________________________________________________

# Attention Mechanism

In [0]:
# from keras.layers import SimpleRNN

encoder = Embedding(english_dict_size, 64, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder_output, state_h, state_c = LSTM(64, return_sequences=True, return_state=True, unroll=True)(encoder)
# encoder_output.shape
# encoder_last = encoder[:,-1,:]

# print('encoder', encoder)
# print('encoder_last', encoder_last)

decoder = Embedding(japanese_dict_size, 64, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder_output,decoder_fwd_state,decoder_back_state = LSTM(64, return_sequences=True, return_state=True, unroll=True)(decoder, initial_state=[state_h, state_c])

# print('decoder', decoder)

In [0]:
from keras.layers import Activation, dot, concatenate

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder_output, encoder_output], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(64, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(output_dict_size, activation="softmax"))(output)
print('output', output)

AttributeError: ignored

In [0]:
# Encoder Input
training_encoder_input = encoded_training_input

# Decoder Input (need padding py START_CHAR_CODE)
training_decoder_input = np.zeros_like(encoded_training_output)
training_decoder_input[:, 1:] = encoded_training_output[:,:-1]
training_decoder_input[:, 0] = START_CHAR_CODE

# Decoder Output (one-hot encode)
training_decoder_output = np.eye(japanese_dict_size)[encoded_training_output.astype('int')]


print('encoder input', training_encoder_input[:1])
print('decoder input', training_decoder_input[:1])
print('decoder output', training_decoder_output[:1].argmax(axis=2))
print('decoder output (one-hot)', training_decoder_output[:1])

encoder input [[36 40 49 40  3 40 38 44 42 29  0  0  0  0  0  0  0  0  0  0]]
decoder input [[ 1 87 86 61 41  8 19  0  0  0  0  0  0  0  0  0  0  0  0  0]]
decoder output [[87 86 61 41  8 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
decoder output (one-hot) [[[0. 0. 0. ... 0. 1. 0.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [0]:

validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)
validation_decoder_input[:, 1:] = encoded_validation_output[:,:-1]
validation_decoder_input[:, 0] = START_CHAR_CODE
validation_decoder_output = np.eye(japanese_dict_size)[encoded_validation_output.astype('int')]

In [0]:
if os.path.isfile('model.h5'):
    model = load_model('model.h5')
else:
    model.fit(x=[training_encoder_input, training_decoder_input], 
              y=[training_decoder_output],
              validation_data=(
                  [validation_encoder_input, validation_decoder_input], 
                  [validation_decoder_output]),
              verbose=2, 
              batch_size=64,
              epochs=20)
model.save('model.h5')

Train on 64356 samples, validate on 10726 samples
Epoch 1/20
64356/64356 - 114s - loss: 1.5116 - val_loss: 1.3812
Epoch 2/20
64356/64356 - 107s - loss: 1.3129 - val_loss: 1.2508
Epoch 3/20
64356/64356 - 107s - loss: 1.1735 - val_loss: 1.1112
Epoch 4/20
64356/64356 - 108s - loss: 1.0575 - val_loss: 1.0024
Epoch 5/20
64356/64356 - 107s - loss: 0.9595 - val_loss: 0.9185
Epoch 6/20
64356/64356 - 107s - loss: 0.8884 - val_loss: 0.8614
Epoch 7/20
64356/64356 - 107s - loss: 0.8349 - val_loss: 0.8119
Epoch 8/20
64356/64356 - 107s - loss: 0.7893 - val_loss: 0.7731
Epoch 9/20
64356/64356 - 107s - loss: 0.7524 - val_loss: 0.7446
Epoch 10/20
64356/64356 - 106s - loss: 0.7230 - val_loss: 0.7175
Epoch 11/20
64356/64356 - 107s - loss: 0.6979 - val_loss: 0.6967
Epoch 12/20
64356/64356 - 107s - loss: 0.6768 - val_loss: 0.6754
Epoch 13/20
64356/64356 - 107s - loss: 0.6585 - val_loss: 0.6610
Epoch 14/20
64356/64356 - 107s - loss: 0.6421 - val_loss: 0.6462
Epoch 15/20
64356/64356 - 107s - loss: 0.6273 - v

In [0]:
def generate_katakana_sequence(text):
    encoder_input = transform(english_encoding, [text.lower()], 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = START_CHAR_CODE
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return decoder_input[0, 1:]

def decode(decoding, sequence):
    text = ''
    for i in sequence:
        if i == 0:
            break
        text += decoding[i]
    return text

def to_katakana(text):
    decoder_output = generate_katakana_sequence(text)
    return decode(japanese_decoding, decoder_output)

In [0]:
common_american_names = ['James', 'John', 'Robert', 'Mary', 'Patricia', 'Linda']
for name in common_american_names:
    print(name, to_katakana(name))

James ジェームズ
John ジョン
Robert ロバート
Mary マリー
Patricia パトリチャ
Linda リンダ
